In [58]:
import pandas as pd
import numpy as np
import re

In [2]:
uid_train = pd.read_csv('../data/uid_train.txt',sep='\t',header=None,names=('uid','label'))
voice_train = pd.read_csv('../data/voice_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})
sms_train = pd.read_csv('../data/sms_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})
wa_train = pd.read_csv('../data/wa_train.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
voice_test = pd.read_csv('../data/voice_test_a.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})
sms_test = pd.read_csv('../data/sms_test_a.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})
wa_test = pd.read_csv('../data/wa_test_a.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

In [4]:
prefix = np.array(['u'])
uid_num = np.arange(5000,7000)
uid_num_char = uid_num.astype('U')
uid_num_str = np.core.defchararray.add(prefix, uid_num_char)
uid_test = pd.DataFrame(uid_num_str, columns=['uid'])
uid_test.to_csv('../data/uid_test_a.txt',index=None)

In [5]:
voice = pd.concat([voice_train,voice_test],axis=0)
sms = pd.concat([sms_train,sms_test],axis=0)
wa = pd.concat([wa_train,wa_test],axis=0)

### 通话记录

In [8]:
voice['voice_dura']=abs(voice.end_time.astype('int')-voice.start_time.astype('int'))
# 不同的电话号码数/电话总数
voice_opp_num = voice.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('voice_opp_num_').reset_index()

# 不同的电话号码头三位的数量
voice_opp_head=voice.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).add_prefix('voice_opp_head_').reset_index()

# 每种电话长度的通话次数
voice_opp_len=voice.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('voice_opp_len_').reset_index().fillna(0)

# 每种类型通话的次数
voice_call_type = voice.groupby(['uid','call_type'])['uid'].count().unstack().add_prefix('voice_call_type_').reset_index().fillna(0)

# 每种类型通话的平均时长
voice_dura_type = voice.groupby(['uid','call_type'])['voice_dura'].mean().unstack().add_prefix('voice_dura_type_').reset_index().fillna(0)

# 接入/打出的电话总数
voice_in_out = voice.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('voice_in_out_').reset_index().fillna(0)

# 通话时长的各统计量
voice_dura = voice.groupby(['uid'])['voice_dura'].agg(['std','max','min','median','mean','sum']).add_prefix('voice_dura_').reset_index().fillna(0)

## 每个用户收/发电话的号码的不同号码数
voice_opp_len_inout = voice.groupby(['uid','in_out'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).unstack().add_prefix('voice_opp_inout_num_').reset_index().fillna(0)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


In [60]:
# 每天通话的次数
day_prefix = np.array(['voice_d'])
day_num = np.arange(1,46)
day_num = day_num.astype('U')
day_num = np.core.defchararray.add(day_prefix, day_num)
day_num = np.concatenate([['uid'],day_num])
day_train = pd.DataFrame(columns=day_num)
day_train.uid = uid_train.uid
day_train.fillna(0,inplace=True)

for i in range(voice.shape[0]):
    #day_train[day_train.uid==voice[i:i+1].uid].voice_d1 = 1
    #print(re.findall(r"\d",voice[i:i+1].uid))

AttributeError: 'Series' object has no attribute 'value'

In [26]:
day_train

AttributeError: 'NoneType' object has no attribute 'describe'

## 短信记录

In [10]:
# 不同的短信号码数/电话总数
sms_opp_num = sms.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('sms_opp_num_').reset_index()

# 号码不同头三位的数量
sms_opp_head=sms.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).add_prefix('sms_opp_head_').reset_index()

# 每种号码长度的通话次数
sms_opp_len=sms.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('sms_opp_len_').reset_index().fillna(0)

# 接受/发出短信总数
sms_in_out = sms.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('sms_in_out_').reset_index().fillna(0)


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """


### 网站/APP记录

In [11]:
# 不同wa数量/wa总数
wa_name = wa.groupby(['uid'])['wa_name'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('wa_name_').reset_index()

# 访问wa次数的各统计量
visit_cnt = wa.groupby(['uid'])['visit_cnt'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_cnt_').reset_index()

# 访问wa时长的各统计量
visit_dura = wa.groupby(['uid'])['visit_dura'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_dura_').reset_index()

# 上传流量的各统计量
up_flow = wa.groupby(['uid'])['up_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_up_flow_').reset_index()

# 下载流量的各统计量
down_flow = wa.groupby(['uid'])['down_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_down_flow_').reset_index()

## 每个用户w/a上传的流量的各统计量
w_a_upflow = wa.groupby(['uid','wa_type'])['up_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('w_a_upflow_').unstack().reset_index().fillna(0)
## 每个用户w/a下载的流量的各统计量
w_a_downflow = wa.groupby(['uid','wa_type'])['down_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('w_a_downflow_').unstack().reset_index().fillna(0)



D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [18]:
test = wa.groupby(['uid','wa_name'])['visit_dura'].sum().unstack().reset_index()
    
test

wa_name,uid,The Merck Group,..::闽南师范大学::..::MINNAN,007卡世界,00934,00小说,021导航网,0460网站之家,06小说手机版手机阅读,07073手游中心,...,龙的天空,龙腾出行,龙腾小说网手机版,龙腾小说网移动版,龙虎网,龙部落,龙部落电影网,龙鱼之巅,ＡＣＲ,Ｑ友吧
0,u0001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,u0002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,u0003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,u0004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,u0005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,u0006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,u0007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,u0008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,u0009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,u0010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# 将各个特征拼在一起
feature = [voice_opp_num,voice_opp_head,voice_opp_len,voice_call_type,voice_in_out,voice_dura,voice_opp_len_inout,voice_dura_type,sms_opp_num,sms_opp_head,sms_opp_len,sms_in_out,wa_name,visit_cnt,visit_dura,up_flow,
           down_flow,w_a_upflow,w_a_downflow]

In [20]:
# 将train特征和test特征分离
train_feature = uid_train
for feat in feature:
    train_feature=pd.merge(train_feature,feat,how='left',on='uid')

D:\Anaconda\lib\site-packages\pandas\core\reshape\merge.py:558: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
D:\Anaconda\lib\site-packages\pandas\core\generic.py:2530: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [21]:
test_feature = uid_test
for feat in feature:
    test_feature=pd.merge(test_feature,feat,how='left',on='uid')

D:\Anaconda\lib\site-packages\pandas\core\reshape\merge.py:558: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
D:\Anaconda\lib\site-packages\pandas\core\generic.py:2530: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [22]:
# 填补Nan
train_feature.fillna(0,inplace=True)
test_feature.fillna(0,inplace=True)

In [23]:
# 保存特征值
train_feature.to_csv('../data/train_featureV1.csv',index=None)
test_feature.to_csv('../data/test_featureV1.csv',index=None)

In [24]:
train_feature.head()

,uid,label,voice_opp_num_unique_count,voice_opp_num_count,voice_opp_head_unique_count,voice_opp_len_3,voice_opp_len_5,voice_opp_len_6,voice_opp_len_7,voice_opp_len_8,...,"(w_a_downflow_max, 0.0)","(w_a_downflow_max, 1.0)","(w_a_downflow_min, 0.0)","(w_a_downflow_min, 1.0)","(w_a_downflow_median, 0.0)","(w_a_downflow_median, 1.0)","(w_a_downflow_mean, 0.0)","(w_a_downflow_mean, 1.0)","(w_a_downflow_sum, 0.0)","(w_a_downflow_sum, 1.0)"
0,u0001,0,22.0,79.0,17.0,0.0,3.0,0.0,0.0,0.0,...,32670956.0,58773144.0,0.0,0.0,9431.0,13088.0,5.127058e+05,2.359752e+06,4.896340e+08,4.719503e+08
1,u0002,0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,15438.0,0.0,0.0,0.0,1656.0,0.0,3.267207e+03,0.000000e+00,9.474900e+04,0.000000e+00
2,u0003,0,15.0,21.0,10.0,0.0,1.0,0.0,0.0,2.0,...,39722861.0,457021859.0,0.0,0.0,5389.5,8130.0,2.360312e+05,5.786216e+06,2.317827e+08,1.070450e+09
3,u0004,0,77.0,254.0,31.0,0.0,1.0,0.0,0.0,12.0,...,513377344.0,94557753.0,0.0,0.0,11194.5,266204.5,1.487322e+06,4.732882e+06,2.040605e+09,1.457728e+09
4,u0005,0,55.0,401.0,28.0,0.0,4.0,0.0,0.0,0.0,...,28858654.0,15441021.0,0.0,0.0,7079.0,5008.5,2.864763e+05,8.140799e+05,1.595673e+08,6.838271e+07
